# Setup

In [ ]:
# imports

# External modules
import sys
import os
import torch
import tqdm
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.models import resnet18
from transformers import AutoImageProcessor, AutoModelForImageClassification
from huggingface_hub import from_pretrained_keras
from tqdm.notebook import tqdm as tqdm
from sklearn.model_selection import train_test_split

# fixing paths
project_root = os.path.abspath("..")
sys.path.append(project_root)

# owned modules
from src.datasets import .
from src.models import FeatureExtractor, ClinicalOutcomePredictor, Adversary
from src.utils import custom_collate_fn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#models
google_vit_huge = "google/vit-huge-patch14-224-in21k"
google_derm = "google/derm-foundation"

# seeds
torch.manual_seed(42)
np.random.seed(42)

# Loading dataset